In [14]:
file_path=r"C:\Users\HP\Downloads\Star_Assignments\Insurance_auto_data.csv"

# Task 1

Cleaned the dataset using basic Python (no pandas/numpy).
- Opened and read the CSV file line by line.
- Removed or replaced missing/null values with None.
- Converted numeric fields to appropriate data types (int or float).
- Stored cleaned data as a list of dictionaries for further analysis. 

In [15]:
with open(file_path,"r")as f:
    raw_lines=f.readlines()

In [16]:
raw_lines[:5]

['CLAIM_ID,CLAIM_DATE,CUSTOMER_ID,CLAIM_AMOUNT,PREMIUM_COLLECTED,PAID_AMOUNT,CITY,REJECTION_REMARKS\n',
 'CLM100021,2025-04-01,CUST14285,10419.0,2198.59,6964.46,PUNE,\n',
 'CLM100013,2025-04-01,CUST26471,42468.0,8982.2,30119.67,GUWAHATI,\n',
 'CLM100099,2025-04-02,CUST29309,55897.0,1861.78,55657.15,GUWAHATI,\n',
 'CLM100044,2025-04-02,CUST30275,71785.0,13154.99,53629.3,PUNE,\n']

In [17]:
def clean_data(file_path):
    with open(file_path,"r")as f:
        lines=f.readlines()
    
    header=lines[0].strip().split(',')
    cleaned_data=[]
    
    for line in lines[1:]:
        values=line.strip().split(',')
        # Skip rows with missing number of columns
        if len(values)!=len(header):
            continue 
        
        cleaned_row=[]
        for value in values:
            cleaned_value=value.strip()
             # Handle empty values
            if cleaned_value=='':
                cleaned_value=None
             # Try to convert to float or int if numeric
            elif cleaned_value.replace('.','',1).isdigit():
                if '.' in cleaned_value:
                    cleaned_value=float(cleaned_value)
                else:
                    cleaned_value=int(cleaned_value)
                
            cleaned_row.append(cleaned_value)
        # Convert list of values to dictionary
        row_dict=dict(zip(header,cleaned_row))
        cleaned_data.append(row_dict)
        
    return cleaned_data


In [18]:
cleaned_insurance_data=clean_data(file_path)

In [19]:
cleaned_insurance_data[:5]

[{'CLAIM_ID': 'CLM100021',
  'CLAIM_DATE': '2025-04-01',
  'CUSTOMER_ID': 'CUST14285',
  'CLAIM_AMOUNT': 10419.0,
  'PREMIUM_COLLECTED': 2198.59,
  'PAID_AMOUNT': 6964.46,
  'CITY': 'PUNE',
  'REJECTION_REMARKS': None},
 {'CLAIM_ID': 'CLM100013',
  'CLAIM_DATE': '2025-04-01',
  'CUSTOMER_ID': 'CUST26471',
  'CLAIM_AMOUNT': 42468.0,
  'PREMIUM_COLLECTED': 8982.2,
  'PAID_AMOUNT': 30119.67,
  'CITY': 'GUWAHATI',
  'REJECTION_REMARKS': None},
 {'CLAIM_ID': 'CLM100099',
  'CLAIM_DATE': '2025-04-02',
  'CUSTOMER_ID': 'CUST29309',
  'CLAIM_AMOUNT': 55897.0,
  'PREMIUM_COLLECTED': 1861.78,
  'PAID_AMOUNT': 55657.15,
  'CITY': 'GUWAHATI',
  'REJECTION_REMARKS': None},
 {'CLAIM_ID': 'CLM100044',
  'CLAIM_DATE': '2025-04-02',
  'CUSTOMER_ID': 'CUST30275',
  'CLAIM_AMOUNT': 71785.0,
  'PREMIUM_COLLECTED': 13154.99,
  'PAID_AMOUNT': 53629.3,
  'CITY': 'PUNE',
  'REJECTION_REMARKS': None},
 {'CLAIM_ID': 'CLM100014',
  'CLAIM_DATE': '2025-04-02',
  'CUSTOMER_ID': 'CUST38169',
  'CLAIM_AMOUNT': 18565

# Task 2

Analyzed claims data to identify a city for potential shutdown.
- Aggregated total claims, premiums, and paid amounts for four target cities.
- Calculated claim settlement ratios.
- Identified Ranchi as the best candidate for closure based on low volume and revenue.

In [20]:
target_cities={'PUNE','KOLKATA','RANCHI','GUWAHATI'}

In [21]:
from collections import defaultdict

In [22]:
def analyze_city_metrics(data):
    city_stats=defaultdict(lambda:{
        'claims_count':0,
        'total_premium':0.0,
        'total_claim_amount':0.0,
        'total_paid_amount':0.0
    })
    for row in data:
        city=row.get('CITY')
        if city not in target_cities:
            continue 
        city_stats[city]['claims_count']+=1
        city_stats[city]['total_premium']+=row.get('PREMIUM_COLLECTED')or 0
        city_stats[city]['total_claim_amount']+=row.get('CLAIM_AMOUNT')or 0
        city_stats[city]['total_paid_amount']+=row.get('PAID_AMOUNT')or 0
        
    return city_stats


In [23]:
city_metrics=analyze_city_metrics(cleaned_insurance_data)

In [24]:
for city, stats in city_metrics.items():
    print(f" {city}")
    print(f"  Total Claims: {stats['claims_count']}")
    print(f"  Total Premium Collected: ₹{stats['total_premium']:.2f}")
    print(f"  Total Claim Amount: ₹{stats['total_claim_amount']:.2f}")
    print(f"  Total Paid Amount: ₹{stats['total_paid_amount']:.2f}")
    print(f"  Claim Settlement Ratio: {(stats['total_paid_amount'] / stats['total_claim_amount'] * 100):.2f}%")
    print("-" * 40)


 PUNE
  Total Claims: 37
  Total Premium Collected: ₹369254.79
  Total Claim Amount: ₹1543919.00
  Total Paid Amount: ₹1093273.28
  Claim Settlement Ratio: 70.81%
----------------------------------------
 GUWAHATI
  Total Claims: 24
  Total Premium Collected: ₹261314.84
  Total Claim Amount: ₹1148030.00
  Total Paid Amount: ₹753189.43
  Claim Settlement Ratio: 65.61%
----------------------------------------
 RANCHI
  Total Claims: 17
  Total Premium Collected: ₹148858.60
  Total Claim Amount: ₹570328.00
  Total Paid Amount: ₹401142.67
  Claim Settlement Ratio: 70.34%
----------------------------------------
 KOLKATA
  Total Claims: 16
  Total Premium Collected: ₹140279.78
  Total Claim Amount: ₹847893.00
  Total Paid Amount: ₹636392.57
  Claim Settlement Ratio: 75.06%
----------------------------------------


# Task3

Fixed and applied a rejection classifier function.
- Debugged the original complex_rejection_classifier function.
- Applied the corrected function to the REJECTION_REMARKS field.
- Created a new column REJECTION_CLASS with the classification result for each row.

In [36]:
#### Dic for mapping
REJECTION_REASONS_MAP = {
    "Fake_document": "Fake_document",
    "Not_Covered": "Not_Covered",
    "Policy_expired": "Policy_expired"
}

##### Function 1 #######
def handle_error(error_message):
    print(f"Error: {error_message}")
    return "Errror" 

#### Function 2 #########
def contains_rejection_reason(rejection_text, reason):
    try:
        if rejection_text and isinstance(rejection_text, str):
            return reason.lower() in rejection_text.lower()  
    except Exceptions as e:
        handle_error(f"Error in contains_rejection_reason: {str(e)}")
        
    return False

####### Function 3 #######
def map_rejection_reason(rejection_text):
    try:
        if rejection_text and isinstance(rejection_text, str):
            for reason, rejection_class in REJECTION_REASONS_MAP.items():
                if contains_rejection_reason(rejection_text, reason):  # Check if reason exists in text
                    return rejection_class
            return "Unknown"  
        else:
            return "No Remark"
    except Exception as e:
        handle_error(f"Error in map_rejection_reason: {str(e)}")
        return "Errror"  
    
######## Function 4 ##########
def complex_rejection_classifier(remark_text):
    try:
        if not isinstance(remark_text, str) or len(remark_text.strip()) == 0:
            return "no remark"

        ##### Check for each rejection reason
        fake_doc = contains_rejection_reason(remark_text, "Fake_document")
        not_covered = contains_rejection_reason(remark_text, "Not_Covered")
        policy_expired = contains_rejection_reason(remark_text, "Policy_expired")

        if fake_doc:
            return "Fake_document"
        elif not_covered:
            return "Not_Covered"
        elif policy_expired:
            return "Policy_expired"
        else:
            ### Unknown or null remarks
            return map_rejection_reason(remark_text)
    except Exception as e:
        handle_error(f"Error in complex_rejection_classifier: {str(e)}")
        return "Errror" 


Apply the classifier to the dataset

In [52]:
for row in cleaned_insurance_data:
    if not isinstance(row, dict):
        continue 
    
    remark=row.get("REJECTION_REMARKS")
    
    if remark is not None and isinstance(remark,str) and remark.strip():
        row["REJECTION_CLASS"]=complex_rejection_classifier(remark)
    else:
        row["REJECTION_CLASS"]="no remark"

In [56]:
[
    {
        "CLAIM_ID":row["CLAIM_ID"],
        "REJECTION_REMARKS": row["REJECTION_REMARKS"],
        "REJECTION_CLASS": row["REJECTION_CLASS"]
    }
    for row in cleaned_insurance_data
    if row["REJECTION_REMARKS"] is not None
][:5]

[{'CLAIM_ID': 'CLM100038',
  'REJECTION_REMARKS': 'Rejection reason: Policy_expired in verification.',
  'REJECTION_CLASS': 'Policy_expired'},
 {'CLAIM_ID': 'CLM100059',
  'REJECTION_REMARKS': 'Policy rejected: Policy_expired noted.',
  'REJECTION_CLASS': 'Policy_expired'},
 {'CLAIM_ID': 'CLM100096',
  'REJECTION_REMARKS': 'Fake_document reason led to rejection.',
  'REJECTION_CLASS': 'Fake_document'},
 {'CLAIM_ID': 'CLM100090',
  'REJECTION_REMARKS': 'Claim denied due to Policy_expired.',
  'REJECTION_CLASS': 'Policy_expired'},
 {'CLAIM_ID': 'CLM100086',
  'REJECTION_REMARKS': 'Case flagged as Policy_expired during audit.',
  'REJECTION_CLASS': 'Policy_expired'}]